This is a first test of aws secrets manager

In [50]:
import boto3
import json
import requests
import openai
from openai import OpenAI
from botocore.exceptions import ClientError

Connect to the aws account to get access to secretmanager's API keys

In [ ]:
#Here we use an IAM role created from the root account
client = boto3.client('secretsmanager')

REDS_test_pw = json.loads(client.get_secret_value(SecretId="PerplexityAPI_1")["SecretString"])["PERPLEXITY_API"]


Perplexity Connection

In [ ]:
#get the secret
PerplexityAPI = json.loads(client.get_secret_value(SecretId="PerplexityAPI_1")["SecretString"])["PERPLEXITY_API"]


#print(PerplexityAPI)

# Perplexity API
client_Perplexity = OpenAI(
    api_key = PerplexityAPI,
    base_url="https://api.perplexity.ai"
)


# Define the messages as shown in the docs
messages = [
    {"role": "system", "content": "Be precise and concise."},
    {"role": "user", "content": "How many stars are there in our galaxy?"}
]

# Make the API call
response = client_Perplexity.chat.completions.create(
    model="llama-3.1-sonar-small-128k-online",
    messages=messages,
)

print(response)

ChatCompletion(id='f22491e3-09b5-42ea-84c4-d87d70fcb857', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Milky Way galaxy is estimated to have between 100 to 400 billion stars[4][5]. More precise estimates suggest approximately 200 billion stars[1][2].', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), delta={'role': 'assistant', 'content': ''})], created=1734498677, model='llama-3.1-sonar-small-128k-online', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=36, prompt_tokens=14, total_tokens=50, completion_tokens_details=None, prompt_tokens_details=None), citations=['https://www.astronomy.com/science/astro-for-kids-how-many-stars-are-there-in-space/', 'https://www.littlepassports.com/blog/space/how-many-stars-are-in-the-universe/', 'https://www.esa.int/Science_Exploration/Space_Science/Herschel/How_many_stars_are_there_in_the_Universe',

chatGPT connection

In [ ]:
gptAPI = json.loads(client.get_secret_value(SecretId="openAI_1")["SecretString"])["openAI_1"] #chatGPT 


#openai package is 1.57.4 as of 12/15/2024

# OpenAI API
client_openAI = OpenAI( 
        api_key = gptAPI,
        base_url = "https://api.openai.com/v1/"
    
)

completion = client_openAI.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "Write a haiku about recursion in programming."
        }
    ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='Calling back with grace,  \nFunctions loop in endless dance,  \nDepths of thought embrace.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


Bedrock connection

In [44]:
import json
import boto3

client = boto3.client('bedrock-runtime', region_name='us-west-2')

# Create a Bedrock client
bedrock = boto3.client(service_name="bedrock")

# Set the model ID
model_id = "us.amazon.nova-lite-v1:0"

# Define the prompt
prompt = "Do you think a foundation model's vision capability would exceed a resnet50 trained model for manufactuing defect identification and reporting? give me the short answer only"

# Format the request payload using the model's native structure
request_body = {
    "schemaVersion": "messages-v1",
    "messages": [
        {"role": "user", "content": [{"text": prompt}]}
    ],
    "system": [
        {"text": "You are helpful "}
    ],
    "inferenceConfig": {
        "max_new_tokens": 500,
        "top_p": 0.9,
        "top_k": 20,
        "temperature": 0.7
    }
}

# Convert the request to JSON
request = json.dumps(request_body)

# Invoke the model
response = client.invoke_model(modelId=model_id, body=request)

# Decode the response
model_response = json.loads(response["body"].read())

# Extract and print the response text
print(model_response)

{'output': {'message': {'content': [{'text': 'Possibly, but it depends on the specific implementation and use case.'}], 'role': 'assistant'}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 38, 'outputTokens': 13, 'totalTokens': 51}}
